<!-- dom:TITLE: Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version -->
# Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version
<!-- dom:AUTHOR: \O yvind Ryan -->
<!-- Author: --> **\O yvind Ryan**

Date: **Jan 18, 2017**

<!-- Externaldocuments: applinalg -->
<!-- Mapping from exercise labels to numbers: label2numbers = {'ex:compareextimes': '2.24', 'dftmainex3': '2.18', 'dftmainex2': '2.17', 'ex:interpolant': '2.21', 'dftmainex': '2.16', 'ex:nonrecursivealg': '2.28'} -->





# Sound and Fourier series
# Digital sound and Discrete Fourier analysis


<!-- --- begin exercise --- -->

## Example 2.16: Using the DFT to adjust frequencies in sound
<div id="dftmainex"></div>
<!-- keywords = ipynbfouriervectors; mfouriervectors -->

In [1]:
%matplotlib inline

import os, sys
sys.path.append(os.path.join(os.getcwd(), 'python'))
from sound import *
from forward_compress_reverse import *
import matplotlib.pyplot as plt
from time import time

Let us test the function `forw_comp_rev_DFT` to listen to the lower frequencies in the audio sample file.
We start with $L=13000$.

In [2]:
x, fs = forw_comp_rev_DFT(L=13000, lower=1)
play(x, fs)

Then with $L=5000$.

In [3]:
x, fs = forw_comp_rev_DFT(L=5000, lower=1)
play(x, fs)

With $L=13000$ you can hear the disturbance in the sound, but we have not lost that much even if about 90\% of the DFT coefficients are dropped. 
The quality is much poorer when $L=5000$ (here we keep less than 5\% of the DFT coefficients). 
However we can still recognize the song, and this suggests that most of the frequency information is contained in the lower frequencies.

Let us then listen to higher frequencies instead. 
We start with $L=140000$.

In [4]:
x, fs = forw_comp_rev_DFT(L=140000, lower=0)
play(x, fs)

Then with $L=100000$.

In [5]:
x, fs = forw_comp_rev_DFT(L=100000, lower=0)
play(x, fs)

We find that we need very high values of $L$ to hear anything, suggesting again that most information is contained in the lowest frequencies.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 2.17: Compression by zeroing out small DFT coefficients
<div id="dftmainex2"></div>
<!-- keywords = ipynbfouriervectors; mfouriervectors -->

We can achieve compression of a sound by setting small DFT coefficients which to zero. The idea is that frequencies with small values at the
corresponding frequency indices contribute little to our perception of the sound, so that they can be discarded. As a result we obtain a sound with less frequency
components, which is thus more suitable for compression. To test this in practice, we first need to set a threshold, which decides which frequencies to keep. 
This can then be sent to the function `forw_comp_rev_DFT` by means of the named parameter `threshold`. 
The function will now also write to the display the percentage of the DFT coefficients which were zeroed out.
Let us first listen to the result when `threshold=20`.

In [6]:
x, fs = forw_comp_rev_DFT(threshold=20)
play(x, fs)

Then for `threshold=70`.

In [7]:
x, fs = forw_comp_rev_DFT(threshold=70)
play(x, fs)

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 2.18: Compression by quantizing DFT coefficients
<div id="dftmainex3"></div>
<!-- keywords = ipynbfouriervectors; mfouriervectors -->

The previous example is a rather simple procedure to obtain compression. The disadvantage is that it only affects frequencies with low contribution. 
A more neutral way to obtain compression is to let each DFT index occupy a certain number of bits. This is also called *quantization*, 
and provides us with compression if the number of bits is less than what actually is used to represent the sound. 
This is closer to what modern audio standards do.
`forw_comp_rev_DFT` accepts a name parameter `n`. The effect of this is that a DFT coefficient with bit representation

$$
...d_2d_1d_0.d_{-1}d_{-2}d_{-3}...
$$

is truncated so that the bits $d_{n-1}$, $d_{n-2}$, $d_{n-2}$ are discarded.
In other words, high values of $n$ mean more rounding. 
Let us first listen to the result for `n=3`.

In [8]:
x, fs = forw_comp_rev_DFT(n=3)
play(x, fs)

Then for `n=5`.

In [9]:
x, fs = forw_comp_rev_DFT(n=5)
play(x, fs)

Then for `n=7`.

In [10]:
x, fs = forw_comp_rev_DFT(n=7)
play(x, fs)

You can hear that the sound degrades further when $n$ is increased.

In practice this quantization procedure is also too simple, since the
human auditory system is more sensitive to certain frequency information, and should thus allocate a higher number of bits for such frequencies. 
Modern audio standards take this into account, but we will not go into details on this.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 2.21: Implement interpolant
<div id="ex:interpolant"></div>
<!-- keywords = ipynbfouriervectors; mfouriervectors -->

Implement code where you do the following:

* at the top you define the function $f(x)=\cos^6(x)$, and $M=3$,

* compute the unique interpolant from $V_{M,T}$ (i.e. by taking $N=2M+1$ samples over one period), 
  as guaranteed by Proposition 2.9,

* plot the interpolant against $f$ over one period.

Finally run the code also for $M=4$, $M=5$, and $M=6$. Explain why the plots coincide for $M=6$, but not for $M<6$. Does increasing $M$ above $M=6$ have any effect on the plots?


<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look as follows.

In [11]:
import matplotlib.pyplot as plt
from numpy import *

f = lambda t:cos(t)**6
M = 5
T = 2*pi
N = 2*M + 1
t = linspace(0, T, 100)
x = f(linspace(0, T - T/float(N), N))
y = fft.fft(x, axis=0)/N
s = real(y[0])*ones(len(t))
for k in range(1,(N+1)/2): 
    s += 2*real(y[k]*exp(2*pi*1j*k*t/float(T)))
plt.plot(t, s, 'r', t, f(t), 'g')
plt.legend(['Interpolant from V_{M,T}','f'])

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 2.24: Compare execution time
<div id="ex:compareextimes"></div>
<!-- keywords = ipynbfouriervectors; mfouriervectors -->

In this exercise we will compare execution times for the different methods for computing the DFT.


**a)**
Write code which compares the execution times for an $N$-point DFT for the following three cases: Direct implementation of the DFT 
(as in Example 2.4),
the FFT implementation used in this chapter, and the built-in `fft`-function. Your code should use the sample audio file `castanets.wav`, 
apply the different DFT implementations to the first $N=2^r$ samples of the file for $r=3$ to $r=15$, store the execution times in a vector, and plot these. You can use 
the function `time()` in the `time` module
to measure the execution time.


**b)**
A problem for large $N$ is that there is such a big difference in the execution times between the two implementations. We can address this by using a loglog-plot instead. Plot $N$ against execution
times using the function `loglog`. How should the fact that the number of arithmetic operations are $8N^2$ and $5N\log_2N$ be reflected in the plot?


<!-- --- begin solution of exercise --- -->
**Solution.**
The two different curves you see should have a derivative approximately equal to one and two, respectively.

<!-- --- end solution of exercise --- -->

**c)**
It seems that the built-in FFT is much faster than our own FFT implementation, even though they may use similar algorithms. Try to explain what can be the cause of this.


<!-- --- begin solution of exercise --- -->
**Solution.**
There may be several reasons for this. 
One is that Python code runs slowly when compared to native code, which is much used in the built-in FFT. Also, the built-in `fft` has been subject to much more optimization than we have
covered here.

<!-- --- end solution of exercise --- -->


<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look as follows.

In [12]:
x0, fs = audioread('sounds/castanets.wav')

kvals = arange(3,16)
slowtime = zeros(len(kvals))
fasttime = zeros(len(kvals))
fastesttime = zeros(len(kvals))
N = 2**kvals
for k in kvals:
    x = x0[0:2**k].astype(complex)
    
    start = time()
    DFTImpl(x)
    slowtime[k - kvals[0]] = time() - start
    
    start = time()
    FFTImpl(x, FFTKernelStandard)
    fasttime[k - kvals[0]] = time() - start

    start = time()
    fft.fft(x, axis=0)
    fastesttime[k - kvals[0]] = time() - start

# a.
plt.plot(kvals, slowtime, 'ro-', \
     kvals, fasttime, 'go-', \
     kvals, fastesttime, 'bo-')
plt.grid('on')
plt.title('time usage of the DFT methods')
plt.legend(['DFT', 'Standard FFT', 'Built-in FFT'])
plt.xlabel('log_2 N')
plt.ylabel('time used [s]')
plt.show()
 
plt.figure()

# b.
plt.loglog(N, slowtime, 'ro-', N, fasttime, 'go-', N, fastesttime, 'bo-')
plt.axis('equal')
plt.legend(['DFT', 'Standard FFT', 'Built-in FFT'])

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 2.28: Non-recursive FFT algorithm
<div id="ex:nonrecursivealg"></div>
<!-- keywords = ipynbfouriervectors; mfouriervectors -->

Use the factorization in (2.18) to write a kernel function `FFTKernelNonrec` 
for a non-recursive FFT implementation.
In your code, perform the matrix multiplications in Equation (2.18) 
from right to left in an (outer) for-loop. For each matrix loop through the different blocks on the diagonal in
an (inner) for-loop. Make sure you have the right number of blocks on the diagonal, each block being on the form

$$
\begin{pmatrix} I & D_{N/2^k} \\ I & -D_{N/2^k} \end{pmatrix}.
$$

It may be a good idea to start by implementing multiplication with such a simple matrix first as these are the building blocks in the algorithm (also attempt to do this so that everything is computed
in-place). Also compare the execution times with our original FFT algorithm, as we did in 
Exercise 2.24, and try to explain what you see in this comparison.







<!-- --- begin solution of exercise --- -->
**Solution.**
The algorithm for the non-recursive FFT can look as follows

In [13]:
def FFTKernelNonrec(x, forward):
    """
    Compute the FFT of x, using a non-recursive FFT algorithm. 
    
    x: a bit-reversed version of the input. Should have only one axis
    forward: Whether the FFT or the IFFT is applied
    """
    N = len(x)
    sign = -1
    if not forward:
        sign = 1
    D = exp(sign*2*pi*1j*arange(float(N/2))/N)
    nextN = 1
    while nextN < N:
        k = 0
        while k < N:
            xe, xo = x[k:(k + nextN)], x[(k + nextN):(k + 2*nextN)]
            xo *= D[0::(N/(2*nextN))]
            x[k:(k+2*nextN)] = concatenate([xe + xo, xe - xo])
            k += 2*nextN
        nextN *= 2

If you add the non-recursive algorithm to the code from Exercise 2.24, you will see that the non-recursive algorithm performs much better. There may be several
reasons for this. First of all, there are no recursive function calls. Secondly, the values in the matrices $D_{N/2}$ are constructed once and for all with
the non-recursive algorithm. Code which compares execution times for the original FFT algorithm, our non-recursive implementation, and the split-radix algorithm of the next
exercise, can look as follows:

In [14]:
x0, fs = audioread('sounds/castanets.wav') 

kvals = arange(3,16)
slowtime = zeros(len(kvals))
fasttime = zeros(len(kvals))
fastesttime = zeros(len(kvals))
N = 2**kvals
for k in kvals:
    x = x0[0:2**k].astype(complex)
    
    start = time()
    FFTImpl(x, FFTKernelStandard)
    slowtime[k - kvals[0]] = time() - start
    
    start = time()
    FFTImpl(x, FFTKernelNonrec)
    fasttime[k - kvals[0]] = time() - start

    start = time()
    FFTImpl(x, FFTKernelSplitradix)
    fastesttime[k - kvals[0]] = time() - start

plt.plot(kvals, slowtime, 'ro-', \
     kvals, fasttime, 'bo-', \
     kvals, fastesttime, 'go-')
plt.grid('on')
plt.title('time usage of the DFT methods')
plt.legend(['Standard FFT algorithm', \
       'Non-recursive FFT', \
       'Split radix FFT'])
plt.xlabel('log_2 N')
plt.ylabel('time used [s]')
plt.show()

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->


# Operations on digital sound: digital filters
# Symmetric filters and the DCT